In [1]:
import org.apache.spark.sql.SparkSession
val sc = SparkSession.builder().getOrCreate()
sc.version

sc = org.apache.spark.sql.SparkSession@711ad88c


2.4.3

In [4]:
/*
단어나 값의 목록을 가지고 있다고 가정하며, RDD를 생성.
*/
val myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple".split(" ")
val words = sc.sparkContext.parallelize(myCollection, 2)

/*
브로드캐스트 선언.
해당 값은 불변성의 값이며, 액션을 실행할 때 클러스터의 모든 노드에 지연 처리 방식으로 복제됨.
suppBroadcast의 value 메서드를 사용해 위 예제에서 브로드캐스트된 supplementalData 값을 참조할 수 있음.
value 메서드는 직렬화된 함수에서 브로드캐스트된 데이터를 직렬화 하지 않아도 접근할 수 있다. (직렬화와 역직렬화에 대한 부하를 크게 줄임)
*/
val supplementalData = Map("Spark" -> 1000, "Definitive" -> 200, "Big" -> -300, "Simple" -> 100)
val suppBroadcast = sc.sparkContext.broadcast(supplementalData)

suppBroadcast.value

/*
아래 예제는 브래드캐스트된 데이터를 사용해 RDD를 변환하는 것.
해당 key가 있다면, 해당 key의 value를 아니면 0으로 채움.
*/
words.map(word => (word, suppBroadcast.value.getOrElse(word, 0)))
    .sortBy(wordPair => wordPair._2)
    .collect()

myCollection = Array(Spark, The, Definitive, Guide, :, Big, Data, Processing, Made, Simple)
words = ParallelCollectionRDD[2] at parallelize at <console>:37
supplementalData = Map(Spark -> 1000, Definitive -> 200, Big -> -300, Simple -> 100)
suppBroadcast = Broadcast(2)


Array((Big,-300), (The,0), (Guide,0), (:,0), (Data,0), (Processing,0), (Made,0), (Simple,100), (Definitive,200), (Spark,1000))

In [ ]:
/*
해당 예제는 Dataset을 이용함.
*/
import sc.implicits._
case class Flight(DEST_COUNTRY_NAME: String, ORIGIN_COUNTRY_NAME: String, count: BigInt)
val flights = sc.read.parquet("../data/flight-data/parquet/2010-summary.parquet").as[Flight]

/*
출발지나 도착지가 중국인 항공편 수를 구하는 어큐물레이터.
아래 예제는 이름이 지정되지 않은 어큐물레이터를 생성함.
*/
import org.apache.spark.util.LongAccumulator
val accUnnamed = new LongAccumulator
val acc = sc.sparkContext.register(accUnnamed)

/*
아래 예제는 이름이 지정된 어큐물레이터를 생성함.
*/
val accChina = new LongAccumulator
val accChina2 = sc.sparkContext.longAccumulator("China")
// OR
sc.sparkContext.register(accChina, "China")

def accChinaFunc(flight_row: Flight) = {
    val destination = flight_row.DEST_COUNTRY_NAME
    val origin = flight_row.ORIGIN_COUNTRY_NAME
    
    if(destination == "China") {
        accChina.add(flight_row.count.toLong)
    }
    if (origin == "China") {
        accChina.add(flight_row.count.toLong)
    }
}

flights.foreach(flight_row => accChinaFunc(flight_row))

In [ ]:
/*
어큐물레이터 직접 정의할 수 있다. (사용자 정의 어큐물레이터))
*/

import scala.collection.mutable.ArrayBuffer
import org.apache.spark.util.AccumulatorV2

val arr = ArrayBuffer[BigInt]()

class EvenAccumulator extends AccumulatorV2[BigInt, BigInt] {
    private var num:BigInt = 0
    def reset(): Unit = {
        this.num = 0
    }
    def add(intValue: BigInt): Unit = {
        if (intValue % 2 == 0) {
            this.num += intValue
        }
    }
    def merge(other: AccumulatorV2[BigInt, BigInt]): Unit = {
        this.num += other.value
    }
    def value():BigInt = {
        this.num
    }
    def copy(): AccumatorV2[BigInt, BigInt] = {
        new EvenAccumulator
    }
    def isZero():Boolean = {
        this.num == 0
    }
}

val acc = new EvenAccumulator
val newAcc = sc.register(acc, "evenAcc")

acc.value
flights.foreach(flight_row => acc.add(flight_row.count))
acc.value